In [ ]:
import sys

sys.path.append("../")
from src.config import BASE_PATH

import pandas as pd

# Combine DFs

In [ ]:
df_0822 = pd.read_excel(
    BASE_PATH / "data" / "raw" / "Malignant Neoplasms of Tongue 2008-2022.xlsx"
)
df_23 = pd.read_excel(
    BASE_PATH / "data" / "raw" / "Malignant Neoplasms of Tongue 2023.xlsx"
)
print(f"2008-2022 Shape: {df_0822.shape}")
print(f"2023 Shape: {df_23.shape}")

In [ ]:
target_cols = [
    "CaseID",
    # Pre-op
    "SEX",
    "RACE_NEW",
    "ETHNICITY_HISPANIC",
    "INOUT",  # IN/out patient
    "Age",
    "AdmYR",
    "ANESTHES",
    "SURGSPEC",
    "ELECTSURG",  # Elective/Urgent surgery
    "HEIGHT",
    "WEIGHT",
    "DIABETES",
    "SMOKE",
    "DYSPNEA",
    "FNSTATUS2",
    "VENTILAT",
    "HXCOPD",
    "ASCITES",
    "HXCHF",
    "HYPERMED",  # Hypertension
    "RENAFAIL",
    "DIALYSIS",
    "DISCANCR",
    "WNDINF",
    "STEROID",
    "WTLOSS",
    "BLEEDDIS",  # Bleed disorder
    "TRANSFUS",
    "PRSEPIS",  # Pre-op sepsis
    "PRALBUM",  # Pre-op albumin
    "PRWBC",  # Pre-op white blood cell
    "ASACLAS",
    # Intra-op
    "OPTIME",
    # Post-op
    "DISCHDEST",
    "YRDEATH",  # Mortality
    "TOTHLOS",  # Total length of hospital stay
    "SUPINFEC",
    "WNDINFD",  # Deep infection
    "ORGSPCSSI",  # Organ space infection
    "DEHIS",  # Dehisence
    "OUPNEUMO",  # Pnemobnia
    "REINTUB",  # Reintubation
    "PULEMBOL",
    "FAILWEAN",  # Ventilator >48
    "RENAINSF",
    "OPRENAFL",  # Post-op Dialysis
    "URNINFEC",  # UTI
    "CNSCVA",  # Stroke
    "CDARREST",
    "CDMI",  # Myocardial infarction
    "OTHBLEED",  # Bleeding
    "OTHDVT",  # DVT
    "OTHSYSEP",  # Post-op Sepsis
    "OTHSESHOCK",  # Septic shock
    "RETURNOR",  # Reoperation
    "READMISSION1",
    "UNPLANNEDREADMISSION1",
    # CPT
    "CPT",
    "OTHERCPT1",
    "OTHERCPT2",
    "OTHERCPT3",
    "OTHERCPT4",
    "OTHERCPT5",
    "OTHERCPT6",
    "OTHERCPT7",
    "OTHERCPT8",
    "OTHERCPT9",
    "OTHERCPT10",
    "CONCPT1",
    "CONCPT2",
    "CONCPT3",
    "CONCPT4",
    "CONCPT5",
    "CONCPT6",
    "CONCPT7",
    "CONCPT8",
    "CONCPT9",
    "CONCPT10",
    "REOPORCPT1",
    # Podiag/ICD
    "PODIAG",  # Completely NULL for 2023
    "PODIAG10",
    "REOPOR1ICD91",  # Completely NULL for 2023
    "REOPOR1ICD101",
]

In [ ]:
# Inconsistent columns accross dfs fixes:
# 'CASETYPE': #ELECTSURG
# WNDINF: NWNDINFD
# DYSPNEA: MAKE unknown column for  (3 entries instead of binary)
# WTLOSS: MAKE unknown column for  (3 entries instead of binary)
# PODIAG: MAKE NULL for
# REOPOR1ICD91: MAKE NULL for


df_23["ELECTSURG"] = df_23["CASETYPE"]
df_23["WNDINF"] = df_23["NWNDINFD"]
df_23["DYSPNEA"] = "Unknown"
df_23["WTLOSS"] = "Unknown"
df_23["PODIAG"] = None
df_23["REOPOR1ICD91"] = None

##Combine
##NOTE: PODIAG and REOPOR1ICD91 all NA in df_23 (still needed for extraction)
df_23 = df_23[target_cols].copy()
df_23.drop(["PODIAG", "REOPOR1ICD91"], axis=1, inplace=True)
df_0822 = df_0822[target_cols].copy()
df_combined = pd.concat([df_23, df_0822], axis=0)
print(df_23.shape)
print(df_0822.shape)
print(df_combined.shape)

# Extract CPT/ICD

In [ ]:
def extract_cols(df, new_cols_df, target_cols_list, cpt=True, filter_cols=None):

    def normalize_cpt(val):
        # Normalize df CPT columns to string without trailing ".0"
        try:
            return str(int(float(val)))
        except (ValueError, TypeError):
            return str(val)

    def normalize_icd(val):
        # Normalize df ICD columns to string without trailing ".0" and upper case
        return str(val).upper()

    #################Create new columns based on cpt codes#################
    if cpt:
        # Convert CPT codes in search space to strings (ICD all strings already)
        for k, v in new_cols_df.items():
            new_cols_df[k] = set(str(int(code)) for code in v)
        normalize_func = normalize_cpt
    else:
        normalize_func = normalize_icd

    # Apply normalization to all target CPT columns
    for col in target_cols_list:
        df[col] = df[col].apply(normalize_func)

    # Check for matches
    for new_col, codes in new_cols_df.items():
        df[new_col] = df[target_cols_list].isin(codes).any(axis=1).astype(int)

    ############ Subset--> only patients with following procedures############
    if filter_cols:
        df_filtered = df[(df[filter_cols] == 1).any(axis=1)].reset_index(drop=True)
        print("Filtered Shape:", df_filtered.shape)
        return df_filtered
    else:
        print("Resulting (Not filtered) Shape:", df.shape)
        return df

CPT

In [ ]:
new_cpt_cols = {
    # Glossectomy Procedures
    "Partial Glossectomy (Hemiglossectomy_Subtotal)": [41120, 41130, 41135],
    "Composite_Extended Glossectomy": [41150, 41153, 41155],
    "Total Glossectomy (Complete Tongue Removal)": [41140, 41145],
    "Excision of Tongue Lesions (Minor)": [
        41110,
        41100,
        41112,
        41113,
        41114,
        41116,
        41105,
    ],
    # Flap and Reconstruction Procedures
    "Local_Regional Tissue Flaps for Oral Cavity Reconstruction": [
        14301,
        14021,
        14302,
        14040,
        14020,
        14041,
        15733,
        15740,
    ],
    "Free Tissue Transfer (Microvascular Free Flaps) and Complex Flap Reconstruction": [
        15732,
        15734,
        15736,
        15738,
        15750,
        20955,
        15756,
        15757,
        15758,
        42894,
        15770,
        20969,
        21215,
        21230,
        20962,
        20902,
        20962,
    ],
    "Skin Autografts for Head and Neck Reconstruction": [
        15100,
        15120,
        15220,
        15240,
        15101,
        15004,
        15200,
        15241,
        15121,
    ],
    "Skin Autografts for Head and Neck Reconstruction": [
        15275,
        15221,
        15271,
        15272,
        15273,
    ],
    # Subsites
    "Neck Dissection and Lymphadenectomy Procedures": [
        38700,
        38720,
        38724,
        38510,
        38500,
        38542,
        31365,
        41135,
        41140,
        41145,
        41153,
        41155,
    ],
    "Alveolar Ridge and Gingival Procedures": [41874, 40845, 40840],
    "Mandibular Resection and Reconstruction Procedures": [
        21198,
        21244,
        21461,
        21045,
        21044,
        21025,
        21196,
        21245,
        21047,
    ],
    "Peripheral Nerve Repair and Neuroplasty": [64716, 64886, 64885, 64864, 64740],
    # Airway and Access Procedures
    "Tracheostomy Procedures": [31600, 31610, 31611, 31603],
    "Gastrostomy and Esophageal Access Procedures": [
        43830,
        49440,
        43030,
        44500,
        44120,
        43832,
    ],
    # Salivary Gland Procedures
    "Submandibular Gland Excision": [42440, 42420, 42450],
    "Parotid Gland Excision": [42415, 42410, 42505],
    # Pharyngeal and Laryngeal Procedures
    "Laryngeal Resection and Reconstruction Procedures": [
        31360,
        31367,
        31599,
        31365,
        31395,
    ],
    "Pharyngeal Resection and Reconstruction Procedures": [
        42890,
        31395,
        42808,
        42892,
        42894,
        42950,
        42953,
        42962,
    ],
    "Tonsillectomy and Tonsillar Region Procedures": [
        42826,
        42842,
        42845,
        42844,
        42961,
        42821,
        42870,
    ],
}

target_cpt_cols = [
    "CPT",
    "OTHERCPT1",
    "OTHERCPT2",
    "OTHERCPT3",
    "OTHERCPT4",
    "OTHERCPT5",
    "OTHERCPT6",
    "OTHERCPT7",
    "OTHERCPT8",
    "OTHERCPT9",
    "OTHERCPT10",
    "CONCPT1",
    "CONCPT2",
    "CONCPT3",
    "CONCPT4",
    "CONCPT5",
    "CONCPT6",
    "CONCPT7",
    "CONCPT8",
    "CONCPT9",
    "CONCPT10",
    "REOPORCPT1",
]

Create new columns based on cpt codes + filter on some of those columns

In [ ]:
cpt_filter_cols = [
    "Partial Glossectomy (Hemiglossectomy_Subtotal)",
    "Composite_Extended Glossectomy",
    "Total Glossectomy (Complete Tongue Removal)",
    "Excision of Tongue Lesions (Minor)",
]

df_cpt_filtered = extract_cols(
    df_combined, new_cpt_cols, target_cpt_cols, cpt=True, filter_cols=cpt_filter_cols
)

In [ ]:
for col in list(new_cpt_cols.keys()):
    print(df_cpt_filtered[col].value_counts())
    print("*" * 30)

# Extract ICD

In [ ]:
new_icd_cols = {
    "Malignant neoplasm of base of tongue": ["C01", "141.0"],
    "Malignant neoplasm of surface of tongue": [
        "C02.0",  # dorsal
        "141.1",  # dorsal
        "C02.2",  # ventral
        "141.3",  # ventral
    ],
    "Malignant neoplasm of border of tongue": ["C02.1", "141.2"],
    "Malignant neoplasm of anterior two-thirds of tongue unspecified": [
        "C02.3",
        "141.4",
    ],
    "Malignant neoplasm of junctional zone of tongue": ["C02.8", "141.5"],
    # "Malignant neoplasm of lingual tonsil": ["C02.4", "141.6"], # This is combined with unspecified
    "Malignant neoplasm of tongue unspecified": [
        "141",
        "141.8",
        "141.9",
        "C02",
        "C02.",
        "C02.9",
        "C02.4",  # OG part of lingual tonsil
        "141.6",  # OG part of lingual tonsil
    ],
}
target_icd_cols = ["PODIAG", "PODIAG10", "REOPOR1ICD91", "REOPOR1ICD101"]

In [ ]:
df_cpt_icd = extract_cols(
    df_cpt_filtered, new_icd_cols, target_icd_cols, cpt=False, filter_cols=None
)

In [ ]:
for col in list(new_icd_cols.keys()):
    print(df_cpt_icd[col].value_counts())
    print("*" * 30)

In [ ]:
# List of mutually exclusive columns
mal_cols = list(new_icd_cols.keys())


# Combine into a single column with the column name where the value is 1
def combine_mutually_exclusive(row):
    for col in mal_cols:
        if row[col] == 1:
            return col
    raise ValueError  # This shouldn't execute


df_cpt_icd["Malignant neoplasm"] = df_cpt_icd.apply(combine_mutually_exclusive, axis=1)
df_cpt_icd.drop(mal_cols, inplace=True, axis=1)

In [ ]:
export_path = BASE_PATH / "data" / "processed" / "CPT_ICD_Extracted_Cleaned.xlsx"
export_path.parent.mkdir(exist_ok=True, parents=True)
df_cpt_icd.to_excel(export_path, index=True)